api : e129c34c5d78c154f63e7f46965cb9da

เป้าหมาย


1.   เอารายชื่อหนังจากข้อมูล Netflix จาก Kaggle
2.   เอารายชื่อหนังของ tmdb มา merge กัน กับข้อมูลชุดแรก ข้อมูลชุดที่จะมี id ของ tmdb
3.   ทำความสะอาดข้อมูล (ซ้ำ null)
4.   เอา id tmdb ที่ได้ไปวิ่งลูปเรียก api ตาม id เขียนลง csv (คัดคอลัมน์บางส่วน)
5.   เอาไป merge กับข้อมูลในข้อ 2. จากนั้นคัดคอลัมน์ที่ต้องการ แล้วเขียนลงไฟล์ csv





In [ ]:
import gzip
import json
import os
import pandas as pd
import requests

from io import BytesIO
from time import sleep

# Netflix and tmdb id


In [ ]:
!gdown 1JuREBI6CBJcnwVYkVe1ISjzlBHLlnFsn

Downloading...
From: https://drive.google.com/uc?id=1JuREBI6CBJcnwVYkVe1ISjzlBHLlnFsn
To: /content/NetFlix.csv.zip
100% 1.23M/1.23M [00:00<00:00, 71.5MB/s]


In [ ]:
!unzip /content/NetFlix.csv.zip

Archive:  /content/NetFlix.csv.zip
  inflating: NetFlix.csv             


In [ ]:
netFlix = pd.read_csv('/content/NetFlix.csv')
netFlix = netFlix[netFlix['type']=='Movie']

In [ ]:
def tmdb_id_url(category):
    year = str(pd.datetime.today().year)
    month = str(pd.datetime.today().month).zfill(2)
    day = str(pd.datetime.today().day - 1).zfill(2)
    return '_'.join([category, 'ids', month, day, year]) + '.' + "json"

id_url = f"http://files.tmdb.org/p/exports/{tmdb_id_url('movie')}.gz"
with gzip.open(BytesIO(requests.get(id_url).content), 'r') as f_open:
  id_list = f_open.readlines()

id_df = pd.DataFrame([json.loads(i) for i in id_list])

<ipython-input-5-027c4cd768de>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  year = str(pd.datetime.today().year)
<ipython-input-5-027c4cd768de>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  month = str(pd.datetime.today().month).zfill(2)
<ipython-input-5-027c4cd768de>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  day = str(pd.datetime.today().day - 1).zfill(2)


In [ ]:
id_df = id_df.drop_duplicates(subset=["original_title"], keep='last')

In [ ]:
df = id_df.merge(netFlix, left_on="original_title", right_on='title')

In [ ]:
net_df = df[['id', "original_title", "director", "release_year", "description"]]

In [ ]:
net_df.dropna(subset=['director'], inplace=True)

<ipython-input-9-3bcc5ca4389d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_df.dropna(subset=['director'], inplace=True)


In [ ]:
net_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3559 entries, 0 to 3646
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              3559 non-null   int64 
 1   original_title  3559 non-null   object
 2   director        3559 non-null   object
 3   release_year    3559 non-null   int64 
 4   description     3559 non-null   object
dtypes: int64(2), object(3)
memory usage: 166.8+ KB


# TMDB API

In [ ]:
API_KEY = 'e129c34c5d78c154f63e7f46965cb9da'

id_list = net_df.id.tolist()

CATEGORY = 'movie'
DOWNLOADS_PER_DISK_WRITE = 40
MAX_DOWNLOADS_PER_SECOND = 4
MAX_ATTEMPTS = 3
RATE_LIMITER_DELAY_SECONDS = 10
RATE_LIMIT_EXCEEDED_STATUS_CODE = 429
SUCCESSFUL_CALL_STATUS_CODE = 200


COLUMNS = ["id",
  "budget",
  "revenue",
  "genres",
  "original_title",
  "runtime",
  "vote_average",
  "vote_count",
  "credits",
  'production_countries']

JSON_COLUMNS = [ "genres",
                  "cast",
                  "production_countries"]

In [ ]:
def was_successful(response):
    return response.status_code == SUCCESSFUL_CALL_STATUS_CODE


def was_rate_limited(response):
    return response.status_code == RATE_LIMIT_EXCEEDED_STATUS_CODE


def make_request(call_url, prior_attempts=0):
    if prior_attempts >= MAX_ATTEMPTS:
        return None
    response = requests.get(call_url)
    if was_rate_limited(response):
        sleep(RATE_LIMITER_DELAY_SECONDS)
    sleep(1 / MAX_DOWNLOADS_PER_SECOND)
    if was_successful(response):
        return response.json()
    else:
        sleep(1)  # attempt to sleep through any intermittent issues
        return make_request(call_url, prior_attempts + 1)


def make_detail_request(entry_id):
    call_url =  f'https://api.themoviedb.org/3/{CATEGORY}/{entry_id}?api_key={API_KEY}&append_to_response=credits,keywords'
    return make_request(call_url)

def export_data(entries):
  if not entries:
    return None
  df = pd.DataFrame(entries)
  df = df[[i for i in df.columns if i in COLUMNS]]
  df['cast'] = df.credits.apply(lambda x:x['cast'])
  df.drop(['credits'], axis=1)
  for column in JSON_COLUMNS:
    df[column] = df[column].apply(json.dumps)

  needs_header = not(os.path.exists(CATEGORY + '_data.csv'))
  df.to_csv(CATEGORY + '_data.csv', index=False, mode='a+', header=needs_header)


def download_movie_data():
  counter = 0
  entries = []
  for id in id_list:
    current_data = make_detail_request(id)
    if not current_data:
      print(f'Failed on move id : {id}')
      continue
    counter +=1
    entries.append(current_data)
    if counter % DOWNLOADS_PER_DISK_WRITE == 0:
      print(f'Finished downloading {counter} entries for {CATEGORY}')
      export_data(entries)
      entries = []
  export_data(entries)
  print(f'Finished downloading for all {counter} entries.')

In [ ]:
download_movie_data()

Finished downloading 40 entries for movie
Finished downloading 80 entries for movie
Finished downloading 120 entries for movie
Finished downloading 160 entries for movie
Finished downloading 200 entries for movie
Finished downloading 240 entries for movie
Finished downloading 280 entries for movie
Finished downloading 320 entries for movie
Finished downloading 360 entries for movie
Finished downloading 400 entries for movie
Finished downloading 440 entries for movie
Finished downloading 480 entries for movie
Finished downloading 520 entries for movie
Finished downloading 560 entries for movie
Finished downloading 600 entries for movie
Finished downloading 640 entries for movie
Finished downloading 680 entries for movie
Finished downloading 720 entries for movie
Finished downloading 760 entries for movie
Finished downloading 800 entries for movie
Finished downloading 840 entries for movie
Finished downloading 880 entries for movie
Finished downloading 920 entries for movie
Finished down

In [ ]:
tmdb_df = pd.read_csv("/content/movie_data.csv")

In [ ]:
ndf = tmdb_df.copy()

In [ ]:
final_data = net_df.merge(ndf, left_on='id', right_on='id')

In [ ]:
final_df = final_data[['id', 'original_title_x', 'director', 'genres', 'description', 'release_year',  'production_countries', 'runtime',  'budget', 'revenue', 'vote_average', 'vote_count', 'cast']]

In [ ]:
final_df = final_df.rename(columns={'original_title_x': 'title'})

In [ ]:
final_df.to_csv('final_movie_data.csv', index=False)